<a href="https://colab.research.google.com/github/vinnytage/opencap-core/blob/main/uni/COLAB/ClimbingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2+ Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=800w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- analyze novel videos
- assemble animals and tracklets
- create quality check plots!

### This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to your own Google Drive.

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub. We also recommend checking out our preprint, which covers the science of maDLC

**Lauer et al 2021:** https://www.biorxiv.org/content/10.1101/2021.04.30.442096v1




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [11]:
#(this will take a few minutes to install all the dependences!)
!apt update && apt install cuda-11-8
!pip install "deeplabcut[tf]"
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [12]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

After the package is installed, please click "restart runtime" if it appears for DLC changes to take effect in your COLAB environment. You will see this button at the output of the cells above ^.

In [13]:
import deeplabcut
import pickle5 as pickle

## Link your Google Drive (with your labeled data):

- This code assumes you locally installed DeepLabCut, created a project, extracted and labeled frames. Be sure to "check Labels" to confirm you are happy with your data. As, these frames are the only thing that is used to train your network. 💪 You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, place your DLC project folder into you Google Drive- i.e., copy the folder named "Project-YourName-TheDate" into Google Drive.

- Then, click run on the cell below to link this notebook to your Google Drive:

In [14]:
#Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Next, edit the few items below, and click run:


In [16]:
# Define paths
config_path = '/content/drive/My Drive/Climbing Project -Vincent, Ali-2024-10-25/config.yaml'
dlc_path = '/content/drive/My Drive/Climbing Project -Vincent, Ali-2024-10-25/dlc-models'

## Create a multi-animal training dataset:

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#create-training-dataset

- please check the text below, edit if needed, and then click run (this can take some time):

## Start training:
This function trains the network for a specific shuffle of the training dataset.
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#train-the-network

In [ ]:
import yaml

pose_config_path = '/content/drive/My Drive/Climbing Project -Vincent, Ali-2024-10-25/dlc-models/iteration-0/Climbing Project Oct25-trainset95shuffle1/train/pose_cfg.yaml'

with open(pose_config_path, 'r') as f:
    pose_cfg = yaml.safe_load(f)

# Optimize for your specific frame size
pose_cfg.update({
    'batch_size': 4,              # Start with 2
    'global_scale': 0.35,         # Reduce more aggressively given the large input size
    'max_input_size': 1000,       # Limit maximum input size
    'scale_jitter_lo': 0.35,      # Adjust scale jittering range
    'scale_jitter_up': 0.85,      # Adjust upper bound
    'mirror': True,
    'display_iters': 100,
    'save_iters': 2000,
    'multi_step': [[0.001, 50000], [0.0005, 100000], [0.0001, 130000]]
})

with open(pose_config_path, 'w') as f:
    yaml.dump(pose_cfg, f)

# Calculate approximate memory usage
original_pixels = 1520 * 2688
scaled_pixels = int(original_pixels * 0.35)  # After global_scale
print(f"Original image pixels: {original_pixels:,}")
print(f"Scaled image pixels: {scaled_pixels:,}")
print(f"Approximate tensor size after scaling: {scaled_pixels * 2 * 4 * 4 / 1024 / 1024:.2f} MB per batch")

# Start training
import tensorflow as tf
tf.keras.backend.clear_session()

deeplabcut.train_network(
    config_path,
    shuffle=1,
    displayiters=100,
    saveiters=2000,
    maxiters=150000,
    gputouse=0,
    allow_growth=True,
    max_snapshots_to_keep=5
)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]],
 'all_joints_names': ['RFH',
                      'RFM',
                      'RFT',
                      'RA',
                      'RK',
                      'LFH',
                      'LFM',
                      'LFT',
                      'LA',
                      'LK',
                      'LeftHold',
                      'RightHold'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 4,
 'contrast': {'clahe': True,
              'claheratio': 0.1,
              'histeq': True,
              'histeqratio': 0.1},
 'convolution': {'edge': False,
                 'emboss': {'alpha': [0.0, 1.0], 'strength': [0.5, 1.5]},
                 'embossratio': 0.1,
                 'sharpen': False,
                 'sharpenratio': 0.3},
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Climbing Project '
            'Oct25/Climbing Project _Vinc

Original image pixels: 4,085,760
Scaled image pixels: 1,430,016
Approximate tensor size after scaling: 43.64 MB per batch
Selecting single-animal trainer
Batch Size is 4
Loading ImageNet-pretrained resnet_101
Max_iters overwritten as 130000
Display_iters overwritten as 100
Save_iters overwritten as 2000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/Climbing Project -Vincent, Ali-2024-10-25/dlc-models/iteration-0/Climbing Project Oct25-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.35, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 4, 'dataset_t

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:

 - First, we evaluate the pose estimation performance.

- This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in a pickle file in the same directory where the video resides.

In [ ]:
print("Start Analyzing my video(s)!")
#EDIT OPTION: which video(s) do you want to analyze? You can pass a path or a folder:
# currently, if you run "as is" it assumes you have a video in the DLC project video folder!

deeplabcut.analyze_videos(path_config_file,videofile_path, shuffle=shuffle, videotype=VideoType)

Optional: Now you have the option to check the raw detections before animals are assembled. To do so, pass a video path:

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
## don't look good, don't proceed with tracking - add more data to your training set and re-train!

#EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/MyDrive/DeepLabCut_maDLC_DemoData/MontBlanc-Daniel-2019-12-16/videos/short.mov'

#don't edit:
deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

If the resulting video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [ ]:
#Check and edit:
numAnimals = 4 #how many animals do you expect to find?
tracktype= 'box' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional:
#imagine you tracked a point that is not useful for assembly,
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

#bodypart= 'Tail_end'
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType,
                                        shuffle=shuffle, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)

Now let's filter the data to remove any small jitter:

In [ ]:
deeplabcut.filterpredictions(path_config_file,
                                 videofile_path,
                                 shuffle=shuffle,
                                 videotype=VideoType,
                                 track_method = tracktype)

## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path,
                                shuffle=shuffle,
                                color_by="individual",
                                videotype=VideoType,
                                save_frames=False,
                                filtered=True)